In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
#from tensorflow import keras
import keras
import matplotlib.pyplot as plt
from vis.visualization import visualize_saliency
from vis.visualization import get_num_filters
from vis.visualization import visualize_activation
from vis.utils import utils
from tensorflow.keras.utils import plot_model
from tensorflow.keras import activations
from tqdm import tqdm
from numba import jit
from joblib import Parallel, delayed

# 1. Loading Data 

## Read dataset

In [ ]:
data = tf.keras.datasets.mnist

In [ ]:
(trainX, trainY),(testX, testY) = data.load_data()

In [ ]:
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [ ]:
trainY = keras.utils.to_categorical(trainY, 10)
testY = keras.utils.to_categorical(testY, 10)

In [ ]:
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

In [ ]:
print(trainX.shape)

# Training

## Code for training

In [ ]:
#Creates Sequential model using Keras
#Number of nodes is the same as number of features (different number of nodes were tried but it did not
#affect validation accuracy significantly)
lenet = keras.Sequential([
                            #Input layer:
                            keras.layers.Conv2D(20, 5, padding="same", input_shape=[28,28,1], use_bias=True),
                            #Hidden Layers:
                            keras.layers.Activation(activation="relu"),
                            keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                            keras.layers.Conv2D(50, 5, padding="same"),
                            keras.layers.Activation(activation="relu"),
                            keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                            keras.layers.Flatten(),
                            keras.layers.Dense(500),
                            keras.layers.Activation(activation="relu"),
                            keras.layers.Dense(10, name='vis',use_bias=True),
                            #Output layer
                            keras.layers.Activation(activation="softmax"),
                        ])

In [ ]:
#lenets = [keras.models.clone_model(lenet),keras.models.clone_model(lenet),keras.models.clone_model(lenet),keras.models.clone_model(lenet),keras.models.clone_model(lenet),keras.models.clone_model(lenet)]

In [ ]:
classifier_number = 5
lenets = [keras.models.clone_model(lenet)]
for i in range(1,classifier_number):
    lenets.append(keras.models.clone_model(lenet))

In [ ]:
#Compiles sequential model
#Using learning rate 0.01
#Loss function will be categorical crossentropy
lenet.compile(
                optimizer=keras.optimizers.SGD(lr=0.01),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy']
                )
#Trains network over a number of epochs and evaluates network agains validation data
#after each epoch
lenetEpochHistory = lenet.fit(trainX, trainY, epochs = 5, validation_data = (testX, testY))

In [ ]:
#Compiles sequential model
#Using learning rate 0.01
#Loss function will be categorical crossentropy
for model in lenets:
    model.compile(
                    optimizer=keras.optimizers.SGD(lr=0.01),
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy']
                    )
#Trains network over a number of epochs and evaluates network agains validation data
#after each epoch
for model in lenets:
    model.fit(trainX, trainY, epochs = 5, validation_data = (testX, testY))

# General Evaluation

In [ ]:
#Predicting Label:

print(np.argmax(lenet.predict(testX)[100]))
print(np.argmax(testY[100]))

In [ ]:
#Get accuracy for lenet
sequentialLoss, sequentialAccuracy = lenet.evaluate(testX, testY)
print('Lenet accuracy: ', sequentialAccuracy)
print('Lenet loss: ', sequentialLoss)

plt.style.use('dark_background')
plt.plot(lenetEpochHistory.history['acc'])
plt.plot(lenetEpochHistory.history['val_acc'])
plt.title('Neural Network accuracy per epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training data', 'Validation data'])
plt.show()

plt.style.use('dark_background')
plt.plot(lenetEpochHistory.history['loss'])
plt.plot(lenetEpochHistory.history['val_loss'])
plt.title('Neural Network loss per epoch')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training data', 'Validation data'])
plt.show()

# Saliency

In [ ]:
#This line outputs the layer_idx that the saliency is meant to be extracted from. (Usually the output layer pre-activation)
utils.find_layer_idx(lenet, 'vis')

In [ ]:
#Check if that layer is correct by checking if the number of nodes matches the number of outputs
get_num_filters(lenet.layers[9])

In [ ]:
#Check if the saliency map is working for the singular network
input_idx = 1
fig, ax = plt.subplots(nrows=1, ncols=2)
saliency_map = visualize_saliency(model = lenet,layer_idx = 9, filter_indices = np.argmax(testY[input_idx]), seed_input = testX[input_idx])
ax[0].imshow(saliency_map.reshape(28,28),interpolation='nearest')
ax[1].imshow(testX[input_idx].reshape(28,28))
print(np.sum(saliency_map))
plt.show()

In [ ]:
#This prints out the pre-activation outputs for the output layer (for curiosity)
intermediate_layer_model = keras.Model(inputs=lenet.input,
                                 outputs=lenet.get_layer("vis").output)
intermediate_output = intermediate_layer_model.predict(testX)[input_idx]
print(max(intermediate_output))
print(lenet.predict(testX)[input_idx])

In [ ]:
#Function to calculate ensemble outputs (vectorised) using mean of outputs
def get_ensenmble_outputs(classifiers,classifier_inputs):
    predictions = np.zeros((np.size(testX, axis=0), np.size(testY, axis=1)))
    for classifier in classifiers:
        predictions = predictions + classifier.predict(classifier_inputs)
    prediction_average = predictions / classifier_number
    outputs = np.apply_along_axis(np.argmax, axis=1, arr=prediction_average)
    return(outputs)
        

In [ ]:
get_ensenmble_outputs(lenets, testX)

In [ ]:
#Function to calculate ensemble output using mean of outputs
def get_ensemble_output(classifiers,classifier_input):
    predictions = np.zeros(np.size(testY, axis=1))
    for classifier in classifiers:
        predictions = predictions + classifier.predict(np.expand_dims(classifier_input,axis=0))
    prediction_average = predictions / classifier_number
    output = np.argmax(prediction_average)
    return(output)
        

In [ ]:
get_ensemble_output(lenets,testX[100])

In [ ]:
#Function to generate multiple saliency maps
def generate_saliency_maps(classifiers,classifier_input,output_node,visualised_layer):
    saliency_maps = np.zeros((len(classifiers),classifier_input.shape[0],classifier_input.shape[1]))
    for i in range(0,len(classifiers)):
        saliency_maps[i] = visualize_saliency(model = classifiers[i],layer_idx = visualised_layer, filter_indices = output_node, seed_input = classifier_input)
    return(saliency_maps)
    

In [ ]:
#Function to visualise the multiple saliency maps
def visualize_saliency_maps(classifier_input,saliency_maps):
    fig, ax = plt.subplots(nrows=1, ncols=len(saliency_maps)+1, figsize = (15,15))
    i = 1
    for s_map in saliency_maps:
        ax[i].imshow(s_map)
        i = i+1
    ax[0].imshow(classifier_input)
    plt.show()

In [ ]:
# Compute difference of saliency maps
def generate_uncertainty_map(saliency_maps):
    return(np.std(saliency_maps,axis=0)/np.average(saliency_maps,axis=0))

In [ ]:
# Wrapper function to arrive at ratio output from classifiers and input
def calculate_uncertainty(classifiers,classifier_input,output_node,visualised_layer):
    
    saliency_maps = generate_saliency_maps(classifiers = classifiers,
                                  classifier_input = classifier_input,
                                  output_node = output_node,
                                  visualised_layer = visualised_layer)
    
    uncertainty_map = generate_uncertainty_map(saliency_maps)
    
    return(np.average(uncertainty_map))


In [ ]:
# Wrapper function to arrive at ratio output from classifiers and input
def calculate_uncertainty(classifiers,classifier_input,output_node,visualised_layer):
    
    saliency_maps = generate_saliency_maps(classifiers = classifiers,
                                  classifier_input = classifier_input,
                                  output_node = output_node,
                                  visualised_layer = visualised_layer)
    
    return(np.average(np.std(saliency_maps,axis=0)/np.average(saliency_maps,axis=0)))

In [ ]:
# Wrapper function to arrive at ratio output from classifiers and input
@jit
def calculate_uncertainty(classifiers,classifier_input,output_node,visualised_layer):
    
    saliency_maps = np.zeros((len(classifiers),classifier_input.shape[0],classifier_input.shape[1]))
    
    for i in range(0,len(classifiers)):
        saliency_maps[i] = visualize_saliency(model = classifiers[i],layer_idx = visualised_layer, filter_indices = output_node, seed_input = classifier_input)
    
    return(np.average(np.std(saliency_maps,axis=0)/np.average(saliency_maps,axis=0)))

In [ ]:
# Wrapper function to arrive at ratio output from classifiers and input
@jit
def calculate_uncertainty(classifiers,classifier_input,output_node,visualised_layer):
    
    saliency_maps = np.zeros((len(classifiers),classifier_input.shape[0],classifier_input.shape[1]))
    
    for i in range(0,len(classifiers)):
        saliency_maps[i] = visualize_saliency(model = classifiers[i],layer_idx = visualised_layer, filter_indices = output_node, seed_input = classifier_input)
    
    return(np.average(np.std(saliency_maps,axis=0)/np.average(saliency_maps,axis=0)))

In [ ]:
# Compute difference of saliency maps
@jit(nopython=True)
def calculate_uncertainty_with_maps(saliency_maps):
    return(np.mean(np.std(saliency_maps,axis=0)/np.average(saliency_maps,axis=0)))

In [ ]:
# Generate saliency maps examples
input_idx = 2
maps = generate_saliency_maps(classifiers = lenets,
                              classifier_input = testX[input_idx],
                              output_node = np.argmax(testY[input_idx]),
                              visualised_layer = 9)

In [ ]:
# Visualise saliency maps examples
visualize_saliency_maps(classifier_input = testX[input_idx].reshape(28,28),
                        saliency_maps = maps)

In [ ]:
for mapx in maps:
    print(np.min(mapx), np.mean(mapx), np.max(mapx))

In [ ]:
# Visualise uncertainty map example
uncertainty_map = generate_uncertainty_map(maps)
plt.imshow(uncertainty_map)
plt.show()

In [ ]:
plt.imshow(np.std(maps,axis=0))
plt.show()

In [ ]:
# Compute the average difference value for each pixel (uncertainty)
np.average(uncertainty_map)

In [ ]:
# Generating example using wrapper function
input_idx = 4
uncertainty = calculate_uncertainty(classifiers = lenets,
                              classifier_input = testX[input_idx],
                              output_node = np.argmax(testY[input_idx]),
                              visualised_layer = 9)
print(uncertainty)

In [ ]:
#Output images that were misclassified by the singular classifier
idx = 0
for i in lenet.predict(testX):
    if (np.argmax(i) != np.argmax(testY[idx])):
        print(idx)
    idx = idx + 1

In [ ]:
uncertainties = np.zeros(testX.shape[0])
#np.size(testX,axis=0)
for input_idx in tqdm(range(0,np.size(testX,axis=0))):
    uncertainties[input_idx] = calculate_uncertainty(classifiers = lenets,
                                               classifier_input = testX[input_idx],
                                               output_node = np.argmax(testY[input_idx]),
                                               visualised_layer = 9)
print(uncertainties)

In [ ]:
for i in range(0,10):
#    visualize_saliency(model = lenet,layer_idx = 9, filter_indices = np.argmax(testY[input_idx]), seed_input = testX[input_idx])
    classifiers = lenets
    classifier_input = testX[input_idx]
    output_node = np.argmax(testY[input_idx])
    visualised_layer = 9
    saliency_maps = np.zeros((len(classifiers),classifier_input.shape[0],classifier_input.shape[1]))
    for i in tqdm(range(0,len(classifiers))):
        saliency_maps[i] = visualize_saliency(model = classifiers[i],layer_idx = visualised_layer, filter_indices = output_node, seed_input = classifier_input)

In [ ]:
input_idx = 8
for i in range(0,10):
#    visualize_saliency(model = lenet,layer_idx = 9, filter_indices = np.argmax(testY[input_idx]), seed_input = testX[input_idx])
    classifiers = lenets
    classifier_input = testX[input_idx]
    output_node = np.argmax(testY[input_idx])
    visualised_layer = 9
    for i in tqdm(range(0,len(classifiers))):
        visualize_saliency(model = classifiers[i],layer_idx = visualised_layer, filter_indices = output_node, seed_input = classifier_input)

In [ ]:
for i in tqdm(range(0,5)):
    visualize_saliency(model = lenet,layer_idx = 9, filter_indices = np.argmax(testY[input_idx]), seed_input = testX[input_idx])


In [ ]:
uncertainties = np.zeros(testX.shape[0])
#np.size(testX,axis=0)
for input_idx in tqdm(range(0,np.size(testX,axis=0))):
    maps = generate_saliency_maps(classifiers = lenets,
                                               classifier_input = testX[input_idx],
                                               output_node = np.argmax(testY[input_idx]),
                                               visualised_layer = 9)
    uncertainties[input_idx] = calculate_uncertainty(maps)
print(uncertainties)

In [ ]:
testX.shape[3]

In [ ]:
classifiers = lenets
saliency_maps = np.zeros((testX.shape[0],
                         len(lenets),
                          testX.shape[1],
                          testX.shape[2]))
for input_idx in tqdm(range(0,np.size(saliency_maps,axis=0))):
    classifier_input = testX[input_idx]
    output_node = np.argmax(testY[input_idx])
    for i in range(0,len(classifiers)):
        saliency_maps[input_idx][i] = visualize_saliency(model = classifiers[i],
                                                         layer_idx = 9, 
                                                         filter_indices = output_node, 
                                                         seed_input = classifier_input)

In [ ]:
def calculate_maps_lambdaish(input_idx):
    print("hello")
    classifier_input = testX[input_idx]
    output_node = np.argmax(testY[input_idx])
    for i in range(0,len(classifiers)):
        saliency_maps[input_idx][i] = visualize_saliency(model = classifiers[i],
                                                         layer_idx = visualised_layer, 
                                                         filter_indices = output_node, 
                                                         seed_input = classifier_input)

In [ ]:
def hello(i):
    print("hello"+str(i))
    return(i*2)

In [ ]:
Parallel(n_jobs=12,verbose=10)(delayed(hello)(i) for i in range(1000000000))

In [ ]:
classifiers = lenets
visualised_layer = 9
saliency_maps = np.zeros((testX.shape[0],
                         len(lenets),
                          testX.shape[1],
                          testX.shape[2]))
saliency_maps = Parallel(n_jobs=2, verbose=10) (
    delayed(calculate_maps_lambdaish) (input_idx)
    for input_idx in range(0,1))

In [ ]:
def calculate_uncertainty_lambdaish(x):
    return(calculate_uncertainty(classifiers = lenets,
                                 classifier_input = x,
                                 output_node = get_ensemble_output(lenets,x),
                                 visualised_layer = 9))

In [ ]:
for x in np.nditer(testX):

In [ ]:
uncertainties = np.zeros(testX.shape[0])

for x in np.nditer(testX):
    uncertainties[i] = calculate_uncertainty(classifiers = lenets,
                                               classifier_input = x,
                                               output_node = get_ensemble_output(lenets,x),
                                               visualised_layer = 9)
    print(done)
print(uncertainties)

uncertainties = np.apply_along_axis(calculate_uncertainty_lambdaish,3,testX)